In [15]:
from pyspark import SparkConf, SparkContext

def load_movies_names():
    movieNames = {}
    with open("/home/jovyan/work/data/ml-100k/u.item", encoding='utf-8',
                 errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

conf = SparkConf().setMaster("local[*]").setAppName("RecommendMovies")
sc = SparkContext.getOrCreate(conf = conf)

nameDict = sc.broadcast(load_movies_names())

lines = sc.textFile("file:///home/jovyan/work/data/ml-100k/u.data")
user_movies_ratings = lines.map(lambda x: x.split()).map(lambda f: (int(f[0]), (int(f[1]), float(f[2]))) )

user_movies_ratings_joins = user_movies_ratings.join(user_movies_ratings).persist()
    
def filter_join(x):
    _, value = x
    c1, c2 = value
    m1, r1 = c1
    m2, r2 = c2
    
    return m1<m2
    
user_movies_ratings_filtered = user_movies_ratings_joins.filter(filter_join)
def squash_to_movies_combos(x):
    _, ((m1,r1), (m2, r2)) = x
    return ((m1,m2), (r1,r2))
movies_combos = user_movies_ratings_filtered.map(squash_to_movies_combos)

movies_ratings_grouped = movies_combos.groupByKey()

movies_combos = movies_ratings_grouped.mapValues(calc_scores)

for result in movies_ratings.take(20):
    print (result)


((242, 580), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233050>)
((242, 692), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b23ae50>)
((242, 428), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233150>)
((242, 340), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b2330d0>)
((393, 1241), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b2331d0>)
((393, 845), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233210>)
((381, 393), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233250>)
((381, 1241), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233110>)
((381, 845), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b2332d0>)
((251, 655), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233310>)
((251, 663), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233350>)
((251, 287), <pyspark.resultiterable.ResultIterable object at 0x7f0d5b233390>)
((251, 1007), <pyspark.resultiterable.ResultIterab